### Cine21 영화배우 정보 웹스크래핑
1. Site 주소 : http://www.cine21.com/rank/person/
2. Request URL : http://www.cine21.com/rank/person/content   
  - 크롬 개발자 도구의 network 탭에서 xhr 인 것만 필터링 
3. Request Method : POST
4. Form Data 
 - section: actor
 - period_start: 2020-09
 - gender: all
 - page: 1
5. 2년치 데이터를 한꺼번에 스크래핑 해서 actors.json 파일로 저장 

In [2]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin


### 1단계 
* 1 page 7명의 이름과 상세정보 추출

### 정규표현식 (regular expression)
* \w는 word를 표현하며 알파벳 + 숫자 + _ 중의 한 문자임을 의미한다.
* 문자가 출현한 횟수를 나타낼때 
 <pre>   
  * : 0 ~ n,  + : 1 ~ n , ? : 0(zero) or 1 
 </pre> 
* 정규표현식에 특수문자를 사용할때는 특수문자 앞에 \(back slash)를 반드시 주어야 한다
* .(period)는 줄바꿈 문자(\n)을 제외한 모든 글자 1개를 의미한다
* Greedy vs Non-Greedy
 - Greedy 욕심많은 (.*) : 모든 태그가 다 선택
 - Non-Greedy 욕심없는 (.*?) : 첫번째 매칭되는 태그만 선택
* [\n\r\t] : \n,\r,\t 중에 하나만 찾는다.
 - re.compile('[\n\r\t]')는 해당 정규표현식을 찾아주는 컴파일러를 생성한다.
 - compile 함수내의 정규표현식 앞에는 항상 r 을 붙혀 주어야 한다.

In [ ]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all',
    'page': 1
}
res = requests.post(req_url, data=req_param)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    
    for li_tag in soup.select('li.people_li'):
        #print(li_tag.select('div.name'))
        div_tag = li_tag.select_one('div.name')
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        name = div_tag.text
        result = re.sub('\(\w*\)','',name)
        #print(result, detail_url)
                       
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                #<li>배우</li> => 배우
                actor_dict_value = re.sub('<.*?>','',result)
                
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub('',actor_dict_value)        
                print(actor_dict_key,actor_dict_value)
                #print(li_tag)

### 2단계 
* 1 page 7명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.

In [ ]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all',
    'page': 1
}
res = requests.post(req_url, data=req_param)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    
    actor_detail_list = []
    actor_li_tag = soup.select('li.people_li')    
    
    for li_tag in actor_li_tag:
        actor_detail_dict = {}
        
        #print(li_tag.select('div.name'))
        div_tag = li_tag.select_one('div.name')
        #상세페이지 url 추출
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        
        #배우이름 추출
        name = div_tag.text
        #성동일(1편) => 성동일 
        actor_name = re.sub('\(\w*\)','',name)
        actor_detail_dict['배우이름'] = actor_name
        
        #순위(ranking) 추출
        #print(li_tag.select_one('span.grade'))                   
        actor_ranking = li_tag.select_one('span.grade').text
        actor_detail_dict['랭킹'] = int(actor_ranking)
        
        #흥행지수 추출
        actor_hit = li_tag.select_one('ul.num_info > li > strong').text
        actor_detail_dict['흥행지수'] = actor_hit.replace(',','')
        
        #상세정보 페이지에 요청을 보냄    
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            
            #기본정보
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                #<li>배우</li> => 배우
                actor_dict_value = re.sub('<.*?>','',result)
                
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub('',actor_dict_value)        
                #print(actor_dict_key,actor_dict_value)
                
                #상세페이지에 추출한 key와 value를 dict에 저장
                actor_detail_dict[actor_dict_key] = actor_dict_value
            
            #참여작품
            movie_title_list = []
            for movie_li_tag in soup.select('ul.part_works li'):
                movie_title = movie_li_tag.select_one('span.tit').text
                movie_title_list.append(movie_title)
            
            #print(movie_title_list)
            actor_detail_dict['출연영화'] = movie_title_list
            
            #상세정보 담은 dict를 list에 추가
            actor_detail_list.append(actor_detail_dict)

print(actor_detail_list[0:2])            

### 3단계 
* 1,2,3 page 21명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.

In [ ]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all'
}

#21명의 배우정보 dict를 저장할 리스트
actor_detail_list = []

for page in range(1,4):
    req_param['page'] = page
    res = requests.post(req_url, data=req_param)
    #print(res.status_code)
    print(f'---> {page} 페이지 출력됨')
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        actor_li_tag = soup.select('li.people_li')
        for idx,li_tag in enumerate(actor_li_tag,1):
            actor_detail_dict = {}

            #print(li_tag.select('div.name'))
            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #배우이름 추출
            name = div_tag.text
            #성동일(1편) => 성동일 
            actor_name = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = actor_name
            
            print(f'======> {idx} : {actor_name}')

            #순위(ranking) 추출
            #print(li_tag.select_one('span.grade'))                   
            actor_ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(actor_ranking)

            #흥행지수 추출
            actor_hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = actor_hit.replace(',','')

            #상세정보 페이지에 요청을 보냄    
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',result)

                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub('',actor_dict_value)        
                    #print(actor_dict_key,actor_dict_value)

                    #상세페이지에 추출한 key와 value를 dict에 저장
                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #참여작품
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)

                #print(movie_title_list)
                actor_detail_dict['출연영화'] = movie_title_list

                #상세정보 담은 dict를 list에 추가
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))                

### 4단계 
* 모든 page의 배우 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.
* itertools 모듈의 count() 함수를 사용하여 무한 Looping
 - 무한 Loop를 탈출할 수 있는 조건
 - actor_li_tag = soup.select('li.people_li') 의 actor_list_tag의 길이가 0 이면 탈출

In [3]:
from itertools import count

req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2018-10',
    'gender': 'all'
}

#21명의 배우정보 dict를 저장할 리스트
actor_detail_list = []

for page in count(1):
    req_param['page'] = page
    res = requests.post(req_url, data=req_param)
    #print(res.status_code)
    if page % 50 == 0:
        print(f'---> {page} 페이지 출력됨')
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        actor_li_tag = soup.select('li.people_li')
        
        #해당 page가 없으면 loop를 탈출
        if len(actor_li_tag) == 0:
            break
            
        for idx,li_tag in enumerate(actor_li_tag,1):
            actor_detail_dict = {}

            #print(li_tag.select('div.name'))
            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #배우이름 추출
            name = div_tag.text
            #성동일(1편) => 성동일 
            actor_name = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = actor_name
            
            print(f'======> {idx} : {actor_name}')

            #순위(ranking) 추출
            #print(li_tag.select_one('span.grade'))                   
            actor_ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(actor_ranking)

            #흥행지수 추출
            actor_hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = actor_hit.replace(',','')

            #상세정보 페이지에 요청을 보냄    
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',result)

                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub('',actor_dict_value)        
                    #print(actor_dict_key,actor_dict_value)

                    #상세페이지에 추출한 key와 value를 dict에 저장
                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #참여작품
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)

                #print(movie_title_list)
                actor_detail_dict['출연영화'] = movie_title_list

                #상세정보 담은 dict를 list에 추가
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))                        

======> 1 : 이병헌
======> 2 : 진선규
======> 3 : 하정우
======> 4 : 마동석
======> 5 : 이하늬
======> 6 : 류승룡
======> 7 : 유해진
======> 1 : 박정민
======> 2 : 황정민
======> 3 : 송강호
======> 4 : 조진웅
======> 5 : 신하균
======> 6 : 조우진
======> 7 : 이동휘
======> 1 : 이정재
======> 2 : 전혜진
======> 3 : 염정아
======> 4 : 공명
======> 5 : 장혜진
======> 6 : 이선균
======> 7 : 유아인
======> 1 : 조정석
======> 2 : 오정세
======> 3 : 이성민
======> 4 : 최우식
======> 5 : 박소담
======> 6 : 김윤석
======> 7 : 공효진
======> 1 : 김의성
======> 2 : 김아중
======> 3 : 김무열
======> 4 : 강기영
======> 5 : 곽도원
======> 6 : 성동일
======> 7 : 김종수
======> 1 : 배수지
======> 2 : 권해효
======> 3 : 임윤아
======> 4 : 주지훈
======> 5 : 이희준
======> 6 : 라미란
======> 7 : 이봉련
======> 1 : 고두심
======> 2 : 장기용
======> 3 : 권상우


KeyboardInterrupt: 

In [4]:
for actor in actor_detail_list:
    if len(actor['배우이름']) > 10:
        print(actor)

In [5]:
print('before', len(actor_detail_list))
actor_detail_final_list = [actor for actor in actor_detail_list \
                           if len(actor['배우이름']) < 10]
print('after', len(actor_detail_final_list))

before 44
after 44


### 5단계
* actor_detail_final_list를 actors.json 파일로 저장하기

In [6]:
import json

with open('data/actors.json','w',encoding='utf8') as file:
    json.dump(actor_detail_final_list, file)

print('json file write 종료')

json file write 종료


In [7]:
with open('data/actors.json','r') as file:
    json_data = json.load(file)

print(len(json_data))    

44
